# pyNBL: Basketball Statistic System for Australian NBL

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ssardina/pynbl/HEAD)

This notebook **incrementally** builds a set of stat tables from NBL Basketball Games:

1. A table of games played, with team names, points, venue, etc.
2. A stat table of _stint lineups_ (advance) statistics for each game and each team. A **stint** is a lineup of players who play together in different interval periods across the game. This table will contain the stints for each team from the play-by-play data and compute various statistics for those stints.

Tables will be saved in CSV and Excel formats as well as in [Pickle format](https://docs.python.org/3/library/pickle.html) for later recovery as Panda DataFrames.


The data comes as a raw JSON file using the game id (e.g., `2087737`):

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [ ]:
# Let's first load all required packages...
import os
from pathlib import Path
import pandas as pd
import numpy as np
import dtale

from nbl.config import *
from nbl import bball_stats, tools


# Set folder with data files and Pickle tables saved on disk
def get_save_files(dir):
    FILES = dict()
    FILES['stint_stats'] = Path(dir, "stint_stats_df").with_suffix('.pkl')
    FILES['stints'] = Path(dir, "stints_df").with_suffix('.pkl')
    FILES['games'] = Path(dir, "games_df").with_suffix('.pkl')
    FILES['players'] = Path(dir, "players_df").with_suffix('.pkl')

    return FILES

## 1. Configuration

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

**Note:** This should be the only section requiring modification.

In [ ]:
# Games to be computed  - Format: (game id, round number)
from games_22_23 import GAMES

# DATA_DIR = 'data-20_21/'
# DATA_DIR = 'data-21_22/'
DATA_DIR = 'data-22_23/'
games = GAMES

games_issues = [('2031329', 0)  # https://github.com/ssardina/pynbl/issues/1
         ]
# DATA_DIR, games = 'test/', games_issues

# Set to True to re-compute from scratch all tables
reload = False

print(f"Games to scrape ({len(games)}):", games)
print("Folder to be used:", DATA_DIR)
print("Reload games?", reload)

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, computes stats and game info, and adds them to the initial tables of stats and games.

We start by loading all saved previous games, if any, as we want to append to that database (and we don't want to recompute them).

In [ ]:
# Load tables from saved files (if any)
saved_stint_stats_df = None
saved_stints_df = None
saved_games_df = None
saved_players_df = None
existing_games = []

FILES = get_save_files(DATA_DIR)

if not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading recorded dataframes from files")
    try:
        saved_stint_stats_df = pd.read_pickle(FILES['stint_stats'])
        saved_stints_df = pd.read_pickle(FILES['stints'])
        saved_games_df = pd.read_pickle(FILES['games'])
        saved_players_df = pd.read_pickle(FILES['players'])
        # collect game ids of all games recovered from file
        existing_games = saved_games_df.game_id.unique()
    except FileNotFoundError as e:
        print("Error loading Pickle files: ", e)
        saved_stint_stats_df = None
        saved_stints_df = None
        saved_games_df = None
        saved_players_df = None
        existing_games = []
else:
    existing_games = []

print(f"Recovered {len(existing_games)} games: ", existing_games)

# saved_stats_df['lineup'].apply(lambda x: len(x) > 5)
# saved_stats_df.loc[5,'lineup']
# saved_stats_df.loc[5]

# saved_stint_stats_df.sample(3)
# saved_games_df.sample(3)

It is now time to process games to extract:

1. Table of **games**.
2. Table of **players** who played in each game with their stats, for each team.
3. Table of **stints** in each game for each team.
4. Table of **stint stats** in each game for each team.

In [ ]:
# collect here set of stat dfs and game info, one per game
#   then, we will put them together into different dataframes
from urllib.error import HTTPError


stint_stats_dfs = []
stints_dfs = []
players_dfs = []
games_data = []

# sort games by rounds (second component of tuple) and get min/max rounds
games.sort(key = lambda x: x[1])
first_round = games[0][1]
last_round = games[-1][1]

# Build data for each game, we'll put them together after....
for game in games:
    # game is beyond the last round available, stop processing..
    if game[1] > last_round:
        break

    # get game_id and round no (if available)
    if isinstance(game, tuple):
        game_id, round_no = game
    else:
        game_id = game
        round_no = np.nan # no round info available

    # don't scrape game data if already loaded from file, skip it
    if game_id in existing_games:
        print(f"Game {game_id} was already saved on file; no scrapping...")
        continue

    ##################################################################
    # !!! MAIN STEP: scrape and compute the actual stats for the game
    ##################################################################
    print(f"Computing game {game_id} (round {round_no})...")

    # 1. Read game JSON file
    try:
        game_json = tools.get_json_data(game_id, dir=DATA_DIR)
    except (HTTPError, ValueError) as e:
        last_round = round_no    # this is the last round to be processed!
        print(f"Game {game_id} JSON data not available yet (last round: {round_no}): ", type(e), e)
        continue

    result = bball_stats.build_game_stints_stats_df(game_json, game_id)
    game_stint_stats_df = result['stint_stats_df']   #  this is basically what we care, the stint stats
    game_stints_df = result['stints_df']
    game_team1, game_team2 = result['teams']

    # Add the game id column to game tables
    game_stint_stats_df.insert(0, 'game_id', game_id)
    game_stints_df.insert(0, 'game_id', game_id)

    # Extract players in the game
    players_df = bball_stats.get_players_stats(game_json)
    players_df.insert(0, 'game_id', game_id)

    # Add tables to collected set of tables, one per game
    stint_stats_dfs.append(game_stint_stats_df)
    stints_dfs.append(game_stints_df)
    players_dfs.append(players_df)

    # Next build the record for the game dataframe
    # first, extract date of game from HTML page
    try:
        game_info = tools.get_game_info(game_id)
    except:
        game_info = { "venue" : np.nan, "date": np.nan}
    print(f"\t .... done: {game_team1[0]} ({game_team1[1]}) vs {game_team2[0]} ({game_team2[1]}) on {game_info['date']}")

    games_data.append({"game_id": game_id,
                        "date" : game_info['date'],
                        "round": round_no,
                        "team1": game_team1[0],
                        "team2": game_team2[0],
                        "s1": game_team1[1],
                        "s2": game_team2[1],
                        "winner": 1 if game_team1[1] > game_team2[1] else 2,
                        "venue" : game_info["venue"]}
                      )


#################################
# All games have been processed, now put all dfs together
#################################
if len(games_data) == 0:
    raise SystemExit("No games!")

# First, build a dataframe with all the game data collected
games_scrapped_df = pd.DataFrame(games_data)    # games that have been scrapped from web
games_df =  games_scrapped_df if saved_games_df is None else pd.concat([saved_games_df, games_scrapped_df])
games_df.reset_index(inplace=True, drop=True)

# Build players dataframe
players_df = pd.concat(players_dfs + ([saved_players_df] if saved_players_df is not None else []))
players_df.reset_index(inplace=True, drop=True)

# Build stint stats dataframe
stint_stats_df = pd.concat(stint_stats_dfs + ([saved_stint_stats_df] if saved_stint_stats_df is not None else []))
stint_stats_df.reset_index(inplace=True, drop=True)

# Build stints dataframe
stints_df = pd.concat(stints_dfs + ([saved_stints_df] if saved_stints_df is not None else []))
stints_df.reset_index(inplace=True, drop=True)

print("Number of total games collected: ", games_df.shape[0])
print("Number of NEW collected: ", games_df.shape[0] - len(existing_games))
print("Number of FAILED games (not yet played): ", len(games) - games_df.shape[0])

# games_df.sample(min(games_df.shape[0], 5))
games_scrapped_df # report all new games scrapped

If we want we can do some sanity checks, before saving to disk:

In [ ]:
# dtale.show(games_df)
# dtale.show(stint_stats_df)
# dtale.show(stints_df)
# dtale.show(players_df)

games_scrapped_df    # show new games scrapped

In [ ]:
import random
print("The shape of stats_df is:", stint_stats_df.shape)
stats_cols = list(stint_stats_df.columns[4:-49])
print("Stats cols:", stats_cols)

# build columns we want to show
cols = ['game_id' , 'tno', 'team', 'stint']
rnd_cols = random.sample(stats_cols, 8)
rnd_cols.extend([f"{x}_opp" for x in rnd_cols])
cols.extend(rnd_cols)

# show some sample of stats computed
stint_stats_df[cols].sample(5)

Sanity check that `(ortg, drtg)` (offensive/defensive rate goal) should mirror `(drtg_opp, ortg_opp)` (opponent offensive/defensive rate goal)):

In [ ]:
# (ortg, drtg) should mirror (drtg_opp, ortg_opp)
stint_stats_df.iloc[4][['game_id' , 'team', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [ ]:
import datetime
import os
import shutil
from pathlib import Path

# make a backup of existing tables on files
for pkl_file in FILES.values():
    for ext in ['.csv', '.xlsx', '.pkl']:
        file = Path(pkl_file).with_suffix(ext)
        if os.path.exists(file):
            # print("Backup file", file)
            shutil.copy(file, str(file) + ".bak")

# dump stint stats dataframe
stint_stats_df.to_pickle(Path(DATA_DIR, "stint_stats_df").with_suffix(".pkl"))
stint_stats_df.to_csv(Path(DATA_DIR, "stint_stats_df").with_suffix(".csv"), index=False)
stint_stats_df.to_excel(Path(DATA_DIR, "stint_stats_df").with_suffix(".xlsx"), index=False)

# dump stint stats dataframe
stints_df.to_pickle(Path(DATA_DIR, "stints_df").with_suffix(".pkl"))
stints_df.to_csv(Path(DATA_DIR, "stints_df").with_suffix(".csv"), index=False)
stints_df.to_excel(Path(DATA_DIR, "stints_df").with_suffix(".xlsx"), index=False)

# dump game dataframe
games_df.to_pickle(Path(DATA_DIR, "games_df").with_suffix(".pkl"))
games_df.to_csv(Path(DATA_DIR, "games_df").with_suffix(".csv"), index=False)
games_df.to_excel(Path(DATA_DIR, "games_df").with_suffix(".xlsx"), index=False)

# dump players dataframe
players_df.to_pickle(Path(DATA_DIR, "players_df").with_suffix(".pkl"))
players_df.to_csv(Path(DATA_DIR, "players_df").with_suffix(".csv"), index=False)
players_df.to_excel(Path(DATA_DIR, "players_df").with_suffix(".xlsx"), index=False)

now = datetime.datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

print(f"Finished saving in {DATA_DIR} @ {date_time}")

## 4. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [ ]:
dtale.show(stint_stats_df)
# dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

## 5. Some checks...

Check if a stint lineup has more than 5 players! It could happen:

1. Game 2031329, player H. Besson comes out (wrongly?) at 3rd period min 10:00 but he keeps playing and then goes out again at 7:33.

In [ ]:
stint_stats_df.shape
mask = stint_stats_df['lineup'].apply(lambda x: len(x) != 5)
stint_stats_df[mask]

# stats_df.iloc[941][['game_id', 'lineup']]